In [46]:
import pandas as pd
data_train = pd.read_csv('Subtask B/trainB.csv')
data_val = pd.read_csv('Subtask B/valB.csv')
data_test = pd.read_csv('Subtask B/Task-B(index,tweet)_label_test.csv')

In [47]:
train_text = data_train['tweet']
val_text = data_val['tweet']
test_text = data_test['tweet']

class_label_train = data_train['label']
class_label_val = data_val['label']
# class_label_test = data_test['label']

In [48]:
class_label_train.value_counts(), class_label_val.value_counts()

(label
 non-hate    16805
 hate         2214
 Name: count, dtype: int64,
 label
 non-hate    3602
 hate         474
 Name: count, dtype: int64)

In [49]:
import numpy as np
classes_list = ["non-hate", "hate"]
label_index_train = class_label_train.apply(classes_list.index)
label_index_val = class_label_val.apply(classes_list.index)

In [50]:
import ktrain
from ktrain import text

In [60]:
# MODEL_NAME = 'google-bert/bert-base-multilingual-cased'
# MODEL_NAME = 'roberta-base'
MODEL_NAME = 'distilbert-base-multilingual-cased'
t = text.Transformer(MODEL_NAME, maxlen=30, classes=classes_list)
trn = t.preprocess_train(np.array(train_text), np.array(class_label_train))
test = t.preprocess_test(np.array(val_text), np.array(class_label_val))
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=test, batch_size=32)

c:\Users\nimmi\anaconda3\Lib\site-packages\ktrain\text\preprocessor.py:382: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn(


preprocessing train...
language: hi
train sequence lengths:
	mean : 22
	95percentile : 44
	99percentile : 50


c:\Users\nimmi\anaconda3\Lib\site-packages\ktrain\utils.py:744: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  warnings.warn(


Is Multi-Label? False
preprocessing test...
language: hi
test sequence lengths:
	mean : 23
	95percentile : 44
	99percentile : 51


In [61]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = "Subtask-B distilbert"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [62]:
learner.fit_onecycle(5e-5, 10, verbose=2, callbacks=callbacks_list) 



begin training using onecycle policy with max lr of 5e-05...
Epoch 1/10

Epoch 1: val_accuracy improved from -inf to 0.88371, saving model to Subtask-B distilbert
595/595 - 4238s - loss: 0.3351 - accuracy: 0.8835 - val_loss: 0.3089 - val_accuracy: 0.8837 - 4238s/epoch - 7s/step
Epoch 2/10

Epoch 2: val_accuracy did not improve from 0.88371
595/595 - 4442s - loss: 0.2990 - accuracy: 0.8834 - val_loss: 0.3031 - val_accuracy: 0.8837 - 4442s/epoch - 7s/step
Epoch 3/10

Epoch 3: val_accuracy did not improve from 0.88371
595/595 - 3153s - loss: 0.2618 - accuracy: 0.8901 - val_loss: 0.3019 - val_accuracy: 0.8830 - 3153s/epoch - 5s/step
Epoch 4/10

Epoch 4: val_accuracy did not improve from 0.88371
595/595 - 2466s - loss: 0.2118 - accuracy: 0.9120 - val_loss: 0.3483 - val_accuracy: 0.8584 - 2466s/epoch - 4s/step
Epoch 5/10

Epoch 5: val_accuracy did not improve from 0.88371
595/595 - 2856s - loss: 0.1643 - accuracy: 0.9334 - val_loss: 0.4630 - val_accuracy: 0.8815 - 2856s/epoch - 5s/step
Epo

In [63]:
filepath = "Subtask-B distilbert"
model.load_weights(filepath)

learner.validate(class_names=t.get_classes())

predictor = ktrain.get_predictor(learner.model, preproc=t)

predict = predictor.predict(test_text.values)

128/128 [==============================] - 156s 1s/step


c:\Users\nimmi\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\nimmi\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\nimmi\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        hate       0.00      0.00      0.00       474
    non-hate       0.88      1.00      0.94      3602

    accuracy                           0.88      4076
   macro avg       0.44      0.50      0.47      4076
weighted avg       0.78      0.88      0.83      4076



In [64]:
labels = [1 if label == 'hate' else 0 for label in predict]

In [65]:
predictions_df = pd.DataFrame({
    'index': data_test['index'],  # Use the index from test_text DataFrame
    'label': labels
})

In [66]:
predictions_df['label'].value_counts()

label
0    4076
Name: count, dtype: int64

In [67]:
predictions_df.to_csv('subtask-B_distilbert_prediction', index=False)